In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, cross_validate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, make_scorer, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np


In [2]:
train = pd.read_csv('data/Train.csv')


In [ ]:
removed_features = ['Place_ID X Date', 'target_min','target_max', 'target_variance','target_count']
# ''',
#                     'L3_AER_AI_sensor_azimuth_angle', 'L3_NO2_sensor_azimuth_angle',
# 'L3_O3_sensor_azimuth_angle', 'L3_CLOUD_sensor_zenith_angle', 'L3_O3_sensor_zenith_angle', 
# 'L3_SO2_SO2_column_number_density', 'L3_AER_AI_sensor_zenith_angle',
# 'L3_SO2_SO2_slant_column_number_density', 'L3_CLOUD_sensor_azimuth_angle', 'L3_NO2_sensor_zenith_angle',
# 'L3_O3_solar_azimuth_angle', 'L3_SO2_sensor_azimuth_angle', 'L3_AER_AI_solar_azimuth_angle', 'L3_CO_sensor_zenith_angle',
# 'L3_CLOUD_cloud_base_pressure']'''

",\n                    'L3_AER_AI_sensor_azimuth_angle', 'L3_NO2_sensor_azimuth_angle',\n'L3_O3_sensor_azimuth_angle', 'L3_CLOUD_sensor_zenith_angle', 'L3_O3_sensor_zenith_angle', \n'L3_SO2_SO2_column_number_density', 'L3_AER_AI_sensor_zenith_angle',\n'L3_SO2_SO2_slant_column_number_density', 'L3_CLOUD_sensor_azimuth_angle', 'L3_NO2_sensor_zenith_angle',\n'L3_O3_solar_azimuth_angle', 'L3_SO2_sensor_azimuth_angle', 'L3_AER_AI_solar_azimuth_angle', 'L3_CO_sensor_zenith_angle',\n'L3_CLOUD_cloud_base_pressure']"

In [4]:
train = train.drop(columns=removed_features)

In [5]:
len(train.columns)

77

In [6]:
train.shape

(30557, 77)

In [ ]:
#train = train[train['target'] <= 300]

In [7]:
target = train['target']
train = train.drop('target', axis=1)

In [8]:
len(train.columns)

76

In [9]:
test = pd.read_csv('data/Test.csv')


In [ ]:
removed_features = ['Place_ID X Date']
# ''', 'L3_AER_AI_sensor_azimuth_angle', 'L3_NO2_sensor_azimuth_angle',
# 'L3_O3_sensor_azimuth_angle', 'L3_CLOUD_sensor_zenith_angle', 'L3_O3_sensor_zenith_angle', 
# 'L3_SO2_SO2_column_number_density', 'L3_AER_AI_sensor_zenith_angle',
# 'L3_SO2_SO2_slant_column_number_density', 'L3_CLOUD_sensor_azimuth_angle', 'L3_NO2_sensor_zenith_angle',
# 'L3_O3_solar_azimuth_angle', 'L3_SO2_sensor_azimuth_angle', 'L3_AER_AI_solar_azimuth_angle', 'L3_CO_sensor_zenith_angle',
# 'L3_CLOUD_cloud_base_pressure']'''

", 'L3_AER_AI_sensor_azimuth_angle', 'L3_NO2_sensor_azimuth_angle',\n'L3_O3_sensor_azimuth_angle', 'L3_CLOUD_sensor_zenith_angle', 'L3_O3_sensor_zenith_angle', \n'L3_SO2_SO2_column_number_density', 'L3_AER_AI_sensor_zenith_angle',\n'L3_SO2_SO2_slant_column_number_density', 'L3_CLOUD_sensor_azimuth_angle', 'L3_NO2_sensor_zenith_angle',\n'L3_O3_solar_azimuth_angle', 'L3_SO2_sensor_azimuth_angle', 'L3_AER_AI_solar_azimuth_angle', 'L3_CO_sensor_zenith_angle',\n'L3_CLOUD_cloud_base_pressure']"

In [11]:
test = test.drop(columns=removed_features)

In [12]:
len(test.columns)

76

In [13]:
train.columns == test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [14]:
train["Date"] = pd.to_datetime(train['Date'])
test["Date"] = pd.to_datetime(test['Date'])

train['month'] = train['Date'].dt.month 
test['month'] = test['Date'].dt.month 

In [15]:
len(train.columns), len(test.columns)

(77, 77)

In [16]:
numeric_cols = train.select_dtypes(include='number').columns
train[numeric_cols] = train.groupby('Place_ID')[numeric_cols].transform(lambda x: x.fillna(x.mean()))

In [17]:
train[numeric_cols] = train[numeric_cols].fillna(train[numeric_cols].mean())


In [18]:
numeric_cols = test.select_dtypes(include='number').columns
test[numeric_cols] = test.groupby('Place_ID')[numeric_cols].transform(lambda x: x.fillna(x.mean()))

In [19]:
New_Features = [
    'precipitable_water_entire_atmosphere','relative_humidity_2m_above_ground',
    'specific_humidity_2m_above_ground', 'temperature_2m_above_ground',
    'L3_NO2_NO2_column_number_density','L3_NO2_NO2_slant_column_number_density','L3_NO2_cloud_fraction',
    'L3_NO2_tropospheric_NO2_column_number_density','L3_SO2_SO2_column_number_density_amf',
    'L3_SO2_absorbing_aerosol_index', 'L3_SO2_cloud_fraction', 'L3_CLOUD_cloud_optical_depth','L3_CLOUD_cloud_top_height', 
    'L3_CLOUD_cloud_top_pressure', 'L3_CLOUD_surface_albedo'
]

for col in New_Features:
    # Group mean per (Place_ID, month)
    group_mean = train.groupby(['Place_ID', 'month'])[col].mean()
    
    # Map to each row in train
    new_col_name = f'mean_{col}_per_month'
    train[new_col_name] = train.set_index(['Place_ID', 'month']).index.map(group_mean)



for col in New_Features:
    # Group mean per (Place_ID, month)
    group_mean = test.groupby(['Place_ID', 'month'])[col].mean()
    
    # Map to each row in train
    new_col_name = f'mean_{col}_per_month'
    test[new_col_name] = test.set_index(['Place_ID', 'month']).index.map(group_mean)


In [20]:
drop_f = ['Date', 'month']
train = train.drop(columns=drop_f)
test = test.drop(columns=drop_f)

In [ ]:
train

In [21]:
numeric_cols = train.select_dtypes(include='number').columns


# Step 1: Detect skewed columns (absolute skewness > 0.75)
skewness = train[numeric_cols].skew()
skewed_cols = skewness[abs(skewness) > 0.75].index.tolist()

# Step 2: Apply PowerTransformer (Yeo-Johnson) to reduce skewness
pt = PowerTransformer(method='yeo-johnson')
train[skewed_cols] = pt.fit_transform(train[skewed_cols])
test[skewed_cols] = pt.transform(test[skewed_cols])

In [22]:
# Select numeric columns
numeric_cols = train.select_dtypes(include='number').columns

# Create a scaler and fit on train
scaler = StandardScaler()
train[numeric_cols] = scaler.fit_transform(train[numeric_cols])

# Apply the same transformation to test
test[numeric_cols] = scaler.transform(test[numeric_cols])

## adding more features

In [23]:
oo = ['Place_ID']

In [24]:
train = train.drop(columns=oo)
test = test.drop(columns=oo)

In [25]:
XGB = XGBRegressor(
    n_estimators=10000,       # Equivalent to 'iterations' in CatBoost
    learning_rate=0.06,      # Same as CatBoost
    max_depth=8,              # Same as 'depth' in CatBoost
    objective='reg:squarederror',  # Default regression objective
    random_state=42,          # Same as 'random_seed' in CatBoost
    subsample=0.7,            # Similar to bagging (adjust as needed)
    colsample_bytree=0.9,     # Feature subsampling
    #early_stopping_rounds=300, # Similar to 'od_wait' (stops if no improvement)
    eval_metric='rmse',       # Same as CatBoost
    n_jobs=-1                 # Use all CPU cores
)

In [26]:
XGB.fit(train, target)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.06, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=10000, n_jobs=-1, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [27]:
y_pred_upload = XGB.predict(test)
dddd = pd.read_csv('data/Test.csv')
dddd['target'] = y_pred_upload
df_upload = dddd[['Place_ID X Date', 'target']]
df_upload.to_csv('data/Submission_20250801_xxxxz.csv', index=False)

In [ ]:
feature_importance_df = pd.DataFrame({
    'Feature': train.columns,
    'Importance': XGB.feature_importances_
})
feature_importance_df.sort_values(by='Importance', ascending=False)


In [ ]:
pipe_XGB.feature_importances_